<a href="https://colab.research.google.com/github/daisysong76/bioinformatics-research/blob/main/lab08_c146_v05_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Data Science for Biology

#### UC Berkeley PMB/MCB/BioE c146, Fall 2022

### Lab 8a: Efficient Algorithms

##### Professor Steven E. Brenner

##### **Developed by**: Xiaomei Song, Nei (Alex) Fang, Amisha Gupta, Ukiah Heasley, Salvador Ramirez Jr

### Table of Contents

1. [Sequences - Part 2](#0) <br>
2. [Dynamic Programming](#1) <br>
3. [Longest Common Substring](#2) <br>
    a. [Brute Force](#3) <br>
    b. [Dot Plot](#4) <br>
    c. [Dynamic Programming](#5) <br>
4. [Summary and Discussion](#6) <br>

In [ ]:
import numpy as np
import timeit
import pandas as pd
from matplotlib import pyplot as plt
import random

# 1. Sequences - Part 2<a id='0'></a>

Recall the **_score_alignment**_ function you implemented in the previous lab. In this implementation, the parameters have default values

**_match_**: +1 point<br>
**_mismatch_**: -1 point<br>
**_gap_**: -1 point<br>
**_endgap_**: 0 points<br>

In Lab 7, you completed function **_score_alignment_**. Copy your function in the cell below. Make sure not to change the values of **match**, **mismatch**, and **gap**.

We have provided the same starter code from Lab 7 as a placeholder. If you are unable to implement the function, we will provide you with our score_alignment function at a later date.

As always, you are not bound to the starter code.

In [ ]:
def score_alignment(string1, string2, match = 1, mismatch = -1, gap = -3, endgap = 0):
    total_score = 0

    # The next two lines calculate the total number of letters in the sequence.
    # Optional: To better understand the code, lookup list comprehension.
    str1_total = len([i for i in string1 if i != '-'])
    str2_total = len([i for i in string2 if i != '-'])

    # These variables keep track of how many letters of the sequence we have checked
    str1_checked = 0
    str2_checked = 0

    for i in range(len(string1)):
        if (string1[i] == string2[i]) and (string1[i] != '-' and string2[i] != '-'):
            total_score += ...
            str1_checked += 1
            str2_checked += 1
        elif (...) and (string1[i] != '-' and string2[i] != '-'):
            ...
        # Why are we checking if str1_checked is 0?
        elif (string1[i] != string2[i]) and (str1_checked == 0 or str2_checked == 0) and (...):
            ...
            if string1[i] != '-':
                ...
            if string2[i] != '-':
                ...
        else:
            ...

    return total_score

As a reminder, two dashes (--) signify a single gap. Let's score:

-- A C G -- -- T

G A C -- -- -- T

In [ ]:
score_alignment('-ACG--T', 'GAC---T')

### Gaps Revisited

Let's think a little bit more about gaps, which we previously penalized equally. Now, we will penalize each gap using the following equation:

**_total_gap_penalty_** = **a** + **_l * b_**

where

**_a_** = penalty for creating the gap<br>
**_l_** = length of the gap<br>
**_b_** = individual gap penalty

For our purposes, we will use the following default values:

_**match**_ = 1, _**mismatch**_ = -1, _**gap_existance**_ = -5, _**individual_gap**_ = -2

## Question 1

###1a.1 Score the following alignment manually using our new gap penalties.

-- A C G -- -- T

G A C -- -- -- T

**_Replace text with your answer_**

###1a.2 Score the following alignment manually using our new gap penalties.

A C G T C A T

-- C T A -- A T

**_Replace text with your answer_**

###1a.3 Score the following alignment manually using our new gap penalties.

A C G T C A T

-- -- C T A A T

**_Replace text with your answer_**

### 1.b Implement the function _alignment_score_affine_gap_. Your function should account for the _total_gap_penalty_ we just introduced.

The function is simply a modification of your previous function already developed in Lab 7. Simply fill in ... with your code. You are not bound to the starter code.

In [ ]:
def alignment_score_affine_gap(string1, string2, match=1, mismatch=-1, gap_existance=-5, individual_gap=-2, endgap=0):
    total_score = 0

    # The next two lines calculate the total number of letters in the sequence.
    # Optional: To better understand the code, lookup list comprehension.
    str1_total = len([i for i in string1 if i != '-'])
    str2_total = len([i for i in string2 if i != '-'])

    # These variables keep track of how many letters of the sequence we have checked
    str1_checked = 0
    str2_checked = 0

    # TODO: YOUR CODE HERE
    # Hint: Have we created a gap yet?
    gap_created = ...

    for i in range(len(string1)):
        if (string1[i] == string2[i]) and (string1[i] != '-' and string2[i] != '-'):
            # TODO: YOUR CODE HERE
            # Hint: Lines 13 - 15
            if gap_created:
                gap_created = ... #  Reset gap flag
            total_score += ...
            str1_checked += 1
            str2_checked += 1
        elif (...) and (string1[i] != '-' and string2[i] != '-'):
            # TODO: YOUR CODE HERE
            # Hint: Lines 13 - 15
            if gap_created:
                gap_created = ...
            ...
        # Why are we checking if str1_checked is 0?
        elif (string1[i] != string2[i]) and (str1_checked == 0 or str2_checked == 0) and (...):
            ...
            if string1[i] != '-':
                ...
            if string2[i] != '-':
                ...
        else:
            # TODO: YOUR CODE HERE
            # Hint: What should we set gap_created to?
            if gap_created == ...:
                gap_created = ...
                total_score += ...
            ...

    return total_score

### 1.c Use your new function to score the alignments and verify the output matches your answers in part 1.1.1.  

In [ ]:
#TEST YOUR CODE
score1 = alignment_score_affine_gap('-ACG--T', 'GAC---T')

In [ ]:
score2 = alignment_score_affine_gap('ACGTCAT', '-CTA-AT')

In [ ]:
score3 = alignment_score_affine_gap('ACGTCAT', '--CTAAT')

In [ ]:
print(f"Score 1: {score1}")
print(f"Score 2: {score2}")
print(f"Score 3: {score3}")

#2. Dynamic Programming<a id='1'></a>

In this section, you'll implement dynamic programming for solving the Longest Common Substring (LCS) problem.

Dynamic programming is a very powerful algorithmic paradigm in which a problem is solved by identifying a collection of subproblems and tackling them one by one, smallest first, using the answers to small problems to help figure out larger ones, until the whole problem is solved.

This aspect of dynamic programming is very similar to how we break a problem into smaller and smaller sub parts when using naive recursion.

**_So, how is it different from naive recursion?_**

Dynamic programming _memoizes_ (not memorizes, although they mean the same thing) the answer to previous sub problems so they do not have to be recomputed. Generally, dynamic programming problems can be split into different subparts. <br>For our future use of dynamic programming for sequences, there are 3 subparts:

_**initialization step**_: Initial setup of a problem, usually an array or a matrix.<br>
_**recursive step**_: Continuous repetition of the steps involved in recursion.<br>
_**traceback step**_: Use of our matrix to go backwards and determine our result.

It is easiest to see this with an example.


Let's take a look at the naive recursive implementation for calculating fibonacci numbers we learned last time. Below is the recursive tree.

![fibonacci](fibtree.png)

In [ ]:
# Recursive implementation
def recursive_fib(fib_number):
    if fib_number == 0:
        return 0
    elif fib_number == 1:
        return 1

    return recursive_fib(fib_number-1) + recursive_fib(fib_number-2)

## Question 2

###In your explanation, describe why the naive recursive implementation for calculating Fibonacci numbers is inefficient.

_**Replace text with your answer.**_

Hopefully, you saw that most of the fibonacci calls are repeated. Take a look at F(2), which appears 3 times in our small call to F(5). This observation is key to understanding how our dynamic programming algorithm will work.

## Question 3

Let's imagine we want to calculate _**Fibonacci(3)**_.

We then need to solve the following subproblems:

_Fibonacci(2)_, _Fibonacci(1)_, _Fibonacci(0)_

At each step, instead of recalculating one of the values, let's just remember the answer to our subproblem and use it in our calculation.

_Where should we store our answers?_ We can just store it in a **list**.

Therefore, to calculate _Fibonacci(3)_, we can simply do **list**[Fibonacci(2)] + **list**[Fibonacci(1)].<br>Then, we add this value to our **list** for future use.

### 3a. Complete the implementation of the function _dp_fib_.

Here's the dp_fib function that efficiently calculates Fibonacci numbers using dynamic programming

In [ ]:
def dp_fib(n):
    # initialization step
    answers = [0,1]

    # recursive step
    if n <= 1:
        return answers[n]

    # traceback step
    for i in range(2, n + 1):
        answers.append(answers[i-1] + answers[i-2])

    return answers[n]

###3b. Verify your answers by comparing the output of dp_fib(30) with the result of recursive_fib(30).

Hint: Verify your answers are the same.

In [ ]:
# Test your code
result_recursive_fib = recursive_fib(30)
result_dp_fib = dp_fib(30)

print(f"Recursive Fibonacci(30): {result_recursive_fib}")
print(f"Dynamic Programming Fibonacci(30): {result_dp_fib}")

Let's take a look at the runtime of our dynamic programming fibonacci function.

In [ ]:
# Do not edit this cell, you don't need to understand how the code works
# This make take a while to run
start_time = timeit.default_timer()
dt = []
for i in range(0, 10000):
    dp_fib(i)
    dt.append(timeit.default_timer()-start_time)
plt.plot(dt)
plt.xlim(5000, 10000)
plt.ylim(5, 25)
plt.xlabel('n')
plt.ylabel('Fibonaci Function Runtime');

### 3C. Recall the complexity of fibonacci using dynamic programming. Did the complexity graph match your expectation?

_**Replace text with your answer.**_

#3. Longest Common Substring (LCS) <a id='2'></a>

In this section, you will address the Longest Common Substring (LCS) problem using different approaches

A _substring_ of a string is a contiguous sequence of characters within a string.

For example, "ACG" and "CG" are substrings of "ACGT". "AGT" is not. A common substring of two strings is a substring that is common to both strings.

_**Example 1**_:

Input: seq1 = "ACGT", seq2 = "CGTA"

Output: "CGT"

Explanation: The longest common substring is "CGT".

_**Example 2**_:

Input: seq1 = "CAT", seq2 = "CT"

Output: "C" or "T"

Explanation: The longest common substring is "C", as it is the first occurance of the longest common substring of length 1. "T" is also techincally valid, but our implementation will always return the first occurance.  

_**Example 3**_:

Input: seq1 = "AGG", seq2 = "TCT"

Output: ""

Explanation: There is no such common substring, so nothing is returned.

## LCS Using a Brute Force Approach<a id='3'></a>

### Slicing

With both lists and strings, you can return a range of entries by using the slice syntax.

Specify the _start index_ (inclusive) and the _end index_ (non-inclusive), separated by a colon, to return a part of the string.

In [ ]:
'ACGT'[0:2]

In [ ]:
[0,1,2,3,4,5][1:4]

## Question 4

Let's implement a brute force way of finding the longest common substring.

To do this, we will generate all possible substring of both strings, and compare them all with each other to see which is the longest common substring.

### 4a. Implement the function get_all_substrings

Hint: which should return a list of all the possible substrings of an input string.

For example:

_**get_all_substrings("ACGT")**_ should return:

['A', 'AC', 'ACG', 'ACGT', 'C', 'CG', 'CGT', 'G', 'GT', 'T']

In [ ]:
def get_all_substrings(string):
    length = len(string)
    alist = []
    for i in range(...):
        for j in range(...,...):
            alist.append(...)
    return alist

print(get_all_substrings('ACGT'))

### 4b. What is the complexity of the function? Explain briefly.

**_Replace text with your answer_**

### 4c. Implement the function lcs_brute_force

Hint: which should return the longest common substring between two strings.

You need to implement the lcs_bruteforce function, which finds the Longest Common Substring between two strings using a brute-force approach.
For example:

_**lcs_brute_force('ATGCGTCAG', 'ATGTGCGTG')**_ should return:

'TGCGT'

_**How should your algorithm work?**_

Create all possible substrings for seq1.<br> Repeat for seq2.<br>

Then, for every subtring of seq1, we must check if it is contained in the possible substrings of seq2.<br>
If it is, we check if the length of the substring is larger then our previous longest substring.

Finally, we return the longest common substring.

In [ ]:
def lcs_brute_force(string1, string2):
    #TODO: generate two lists of substrings
    ...
    ...

    length = 0
    longest = ''
    for i in ...:
        for j in ...:
            if i == j:
                if len(i) > ...:
                    length = ...
                    longest = ...

    return ...

In [ ]:
lcs_brute_force('', '')

In [ ]:
## TEST YOUR CODE: Should return TGCGT
lcs_brute_force('ATGCGTCAG', 'ATGTGCGTG')

As a reminder of something we covered in Lab 7, we can use _cell magic_ and the _timeit_ library to time the execution of Python code.

In [ ]:
# TEST YIOUR CODE: Should return GGGCAT
testString1 = 'TTGCGGGGCATAGCTACGACTTGCTTAGCTACGTGCGAGGGAAGAAACTTTTGCGTATTTGTATGTTCACCCGTCTACTACCCATGCCCGGAGATTATGTAGGTTGTGAGATGCGGGAGAAGTTCTCGACCTTCCCGTGG'
testString2 = 'GACGTCAACCTATCCCTTAATAGAGCATTCCGTTCGGGCATGGCAGTAAGTACGCCTTCTCAATTGTGCTAACCTTCATCCTTATCAAAGCTTGGAGCCAATGATCAGGATTATTGCCTTGCGACAGACTTCCTACTCAC'
%timeit lcs_brute_force(testString1, testString2)
lcs_brute_force(testString1, testString2)

### 4d. What is the complexity of the brute force algorithm we developed, relative to the length of the strings (assume they are both length n)? Explain briefly.

_**Replace text with your answer**_.

###4e.Implement the plot_lcs_runtime function.

You'll need to implement the plot_lcs_runtime function that measures the runtime of lcs_bruteforce for strings of varying lengths and plots the results.

In [ ]:
def plot_lcs_runtime():
    lengths = [5, 10, 15, 20, 25]
    times = []
    for length in lengths:
        str1 = 'ACGT' * (length // 4)
        str2 = 'TGCA' * (length // 4)
        start_time = timeit.default_timer()
        lcs_bruteforce(str1, str2)
        end_time = timeit.default_timer()
        times.append(end_time - start_time)

    plt.plot(lengths, times, marker='o')
    plt.xlabel('String Length')
    plt.ylabel('Runtime (s)')
    plt.title('LCS Bruteforce Runtime')
    plt.show()

## LCS Using Dot Plots<a id='4'></a>

Recall our dot plot code from the previous lab.

In [ ]:
def makeMatrix(seqx,seqy):
    matrix = []
    for i in seqx:
        appendArray = []
        for j in seqy:
            if i == j:
                appendArray.append(1)
            else:
                appendArray.append(0)
        matrix.append(appendArray)
    return matrix

def dotplot(seqx, seqy):
    dotplot = plt.imshow(np.logical_not(makeMatrix(seqy,seqx)).astype(int))
    xt = plt.xticks(np.arange(len(list(seqx))),list(seqx))
    yt = plt.yticks(np.arange(len(list(seqy))),list(seqy))
    plt.show()

Let's take a look at the dot plot for two small strings.

In [ ]:
dotplot('ATGCGTCAG', 'ATGTGCGTG')

We can use the diagonals of the dotplot to determine what counts as a common substring between two sequences.<br> Take a look at the dotplot to convince yourself that this is the case for the the sequence 'ATG'.

Recall that our _makeMatrix_ function generates a 2d array of 0s and 1s, where each subarray is a column in the dotplot.

In [ ]:
makeMatrix('ATGCGTCAG', 'ATGTGCGTG')

To make it more visually similar to the dotplot, let's transpose the array.

In [ ]:
np.transpose(makeMatrix('ATGCGTCAG', 'ATGTGCGTG'))

As you can see, the longest common substring is contained along the diagonal with the most 1s. Therefore, our goal will be to find the diagonal with the most number of 1s.

_**How should your algorithm work?**_

For every element in the matric, we check whether the value is 0 or 1.<br>If it is 0, we move on. <br> If it is 1:
<br>&emsp; we add that character of the string to our current substring.
<br>&emsp; we check the diagonal above and to the left. If it is 1, repeat the above step.
<br>&emsp; If it is 0, we check whether or no the current substring is larger than the current largest susbtring. If it is, we replace our current longest substring.<br>
Finally, we return the longest common substring.

## Question 5

### 5a. Implement lcs_dotplot, which takes in a matrix like the one above and generates the longest common substring.

In [ ]:
def lcs_dotplot(string1, string2):
    matrix = np.transpose(makeMatrix(string1, string2))
    lcs = ''

    #Reminder, we transposed the two strings. string1 used to be a column, now it is a row of matrix
    for i in range(len(...)):
        length = 0
        for j in range(len(...)):
            diagonal = False
            current_i = i
            current_j = j
            if matrix[...][...] == 1:
                diagonal = ...
                current_string = ''
                current_string += ...[...]
                while diagonal:
                    if (current_j - 1 >= ...) & (... >= ...):
                        if matrix[current_i - 1][...] == 1:
                            current_i -= ...
                            current_j -= ...
                            current_string += string2[...]
                        else:
                            diagonal = ...
                    else:
                        diagonal = ...
                if len(...) > len(...):
                    ...

    return lcs[::-1] #This inverts the string, since we added it backwards

In [ ]:
lcs_dotplot('', '')

In [ ]:
lcs_dotplot('ATGCGTCAG', 'ATGTGCGTG')

In [ ]:
# TEST YIOUR CODE: Should return GGGCAT
testString1 = 'TTGCGGGGCATAGCTACGACTTGCTTAGCTACGTGCGAGGGAAGAAACTTTTGCGTATTTGTATGTTCACCCGTCTACTACCCATGCCCGGAGATTATGTAGGTTGTGAGATGCGGGAGAAGTTCTCGACCTTCCCGTGG'
testString2 = 'GACGTCAACCTATCCCTTAATAGAGCATTCCGTTCGGGCATGGCAGTAAGTACGCCTTCTCAATTGTGCTAACCTTCATCCTTATCAAAGCTTGGAGCCAATGATCAGGATTATTGCCTTGCGACAGACTTCCTACTCAC'
%timeit lcs_dotplot(testString1, testString2)
lcs_dotplot(testString1, testString2)

## LCS Using Dynamic Programming<a id='5'></a>
In this section, you will use dynamic programming to code an algorithm for finding the longest common substring in Python.

We are creating one extra row and one extra column. To start, we fill the first row and the first column of the matrix with 0s.

Then, similar to the dot plot implementation, we move along the matrix in order and check if the two characters match. If they match, we set the value of our current cell as the value of the previous diagonal (up and to the left) + 1. In our first case, the two 'A's match, so we add 1 to 0.

![lcsboard](lcs-final-1.png)

Now, we traverse to the cell [1,2], where the characters do not a match. We fill the cell with a 0. We move on to [1,3] because we still needs to check if we can find a match for the next characters.

**Now we see a traversal pattern**:

Our initial matrix is filled with 0s in the first row and first column.

We start off at [i, j], where i = j = 1.

If we find a match, we update that cell with the length of the previous diaognal + 1 and go to [i, j+1].

Else we update the value with 0 and move on.

Our function code implementation keeps track of the indices i and j that have the maximum value in the matrix. However, this is only an efficiency move, and is not necessary. <br>
If we do not do this, we would have to iterate through the matrix again until we found the maximal value.

![lcstraversal](lcs-final-2.png)

## Question 6

### 6a. Implement the _lcs_dp_ function below.


In [ ]:
def lcs_dp(string1, string2):
    length1 = len(string1)
    length2 = len(string2)

    # Which of the 3 subparts of dynamic programming is this?
    matrix = [[0 for k in range(length2 + 1)] if l == 0 else [0] for l in range(length1 +1)]

    # To store the length of
    # longest common substring
    max_i = 0
    max_j = 0
    lcs = ''

    # Which of the 3 subparts of dynamic programming is this?
    for i in range(1, ...):
        current_string = ''
        for j in range(1, ...):
            if (string1[i-1] == ...):
                matrix[i].append(...)
                if matrix[i][j] > matrix[max_i][max_j]:
                    max_i = ...
                    max_j = ...
            else:
                matrix[i].append(...)

    # Which of the 3 subparts of dynamic programming is this?
    while matrix[max_i][max_j] > ...:
        #Which character should we add to lcs? Hint, use max_i.
        lcs += string1[...]

        # move diagonally up to previous cell
        max_i -= ...
        max_j -= ...

    return lcs[::-1]

In [ ]:
lcs_dp('','')

In [ ]:
lcs_dp('ATGCGTCAG', 'ATGTGCGTG')

In [ ]:
# TEST YIOUR CODE: Should return GGGCAT
testString1 = 'TTGCGGGGCATAGCTACGACTTGCTTAGCTACGTGCGAGGGAAGAAACTTTTGCGTATTTGTATGTTCACCCGTCTACTACCCATGCCCGGAGATTATGTAGGTTGTGAGATGCGGGAGAAGTTCTCGACCTTCCCGTGG'
testString2 = 'GACGTCAACCTATCCCTTAATAGAGCATTCCGTTCGGGCATGGCAGTAAGTACGCCTTCTCAATTGTGCTAACCTTCATCCTTATCAAAGCTTGGAGCCAATGATCAGGATTATTGCCTTGCGACAGACTTCCTACTCAC'
%timeit lcs_dp(testString1, testString2)
lcs_dp(testString1, testString2)

### 6b. On which line does the initialization step begin? On which line does the recursion step begin? On which line does the traceback step begin? Explain briefly.

_**Replace text with your answer**_.

### 6c. What is the complexity, in big O notation, for this solution to the LCS problem? What is the complexity of each component (initalization, recursion, traceback)? Your Big O notation answer should include n (the length of both strings). Explain briefly.

_**Replace text with your answer**_.

Now, let's use timeit to plot the runtime of our 3 LCS implementations.

In the following lines of code, we generate 120 random sequences of length 0 - 120. Then, we apply your 3 implementations of LCS.

In [ ]:
dt_bruteforce = []
dt_dotplot = []
dt_dp = []

sequences1 = []
sequences2 = []

for i in range(0, 120):
    sequences1.append(''.join(random.choice('CGTA') for x in range(i)))
    sequences2.append(''.join(random.choice('CGTA') for x in range(i)))

In [ ]:
for i in range(0, 120):
    bruteforce_start = timeit.default_timer()
    lcs_brute_force(sequences1[i], sequences2[i])
    dt_bruteforce.append(timeit.default_timer()-bruteforce_start)

plt.plot(dt_bruteforce)
plt.xlabel('n')
plt.ylabel('Complexity')
plt.title('LCS Brute Force');

In [ ]:
for i in range(0, 120):
    dotplot_start = timeit.default_timer()
    lcs_dotplot(sequences1[i], sequences2[i])
    dt_dotplot.append(timeit.default_timer()-dotplot_start)

plt.plot(dt_dotplot)
plt.xlabel('n')
plt.ylabel('Complexity')
plt.title('LCS Dot Plot');

In [ ]:
for i in range(0, 120):
    dp_start = timeit.default_timer()
    lcs_dp(sequences1[i], sequences2[i])
    dt_dp.append(timeit.default_timer()-dp_start)

plt.plot(dt_dp)
plt.xlabel('n')
plt.ylabel('Complexity')
plt.title('LCS Dynamic Programming');

### 6d. Taking a look at all 3 plots, and the y-axis, which LCS implementation is the slowest? Which is the fastest? Did this match your expectations?

_**Replace text with your answer**_

#4. Summary and Discussion
In this section, provide a summary and discussion of your findings. Discuss the differences in runtime and efficiency between the brute-force and dynamic programming approaches for finding the Longest Common Substring. Highlight the advantages of the dynamic programming approach, especially for larger input strings.

**Summary**:

**Brute-Force Approach**: The brute-force approach for finding the Longest Common Substring involved generating all possible substrings of both input strings and comparing them to find the longest common substring. It had a time complexity of O(n^4), where 'n' is the length of the input strings. This approach was inefficient, especially for longer input strings, as it involved checking a large number of substrings.
**Dynamic Programming Approach**: The dynamic programming approach, implemented in the lcs_dynamic_programming function, significantly improved the efficiency of finding the Longest Common Substring. This method used a 2D table to store the lengths of LCS for different substrings of the input strings. By using a bottom-up dynamic programming approach, it computed the LCS efficiently. The time complexity of this approach is O(m * n), where 'm' and 'n' are the lengths of the two input strings. This approach offers a substantial improvement in performance compared to the brute-force method.

**Discussion**:

**Efficiency**: The most notable difference between the two approaches is their efficiency. The dynamic programming approach is significantly more efficient, especially when dealing with longer input strings. The time complexity of O(m * n) scales well with the size of the input, making it a practical solution for a wide range of input lengths.

**Practicality**: The dynamic programming approach is practical for real-world applications where efficiency matters. For example, in bioinformatics, finding the Longest Common Substring between DNA sequences can be a computationally intensive task. The dynamic programming approach is well-suited for such scenarios.

**Scalability**: The dynamic programming approach's time complexity grows linearly with the input size, making it a scalable solution. In contrast, the brute-force method quickly becomes impractical as the input strings grow larger due to its high combinatorial time complexity.

**Advantages**: The dynamic programming approach offers advantages such as speed, efficiency, and scalability, making it a go-to choice for solving LCS problems in practice. It also has a clear and straightforward implementation, making it accessible to a wide range of users.

After implementing these sections, your lab document will be complete. If you have any more questions or need further assistance, please feel free to ask!
